In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np

#### Import model, training function 

In [3]:
from annsa.load_dataset import dataset_to_spectrakeys
from annsa.model_classes import build_dnn_model, compile_model
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Dataset Construction

#### Load dataset

In [17]:
dataset = np.load(os.path.join('dataset_generation',
                               'uenrichment_dataset_100000.npy'))
training_spectra = np.array(dataset.item()['spectra'], dtype='float64')
all_keys = np.array(dataset.item()['keys'])

## Model Construction

In [18]:
architecture_id = os.path.join('..',
                               'source-interdiction',
                               'final_training_notebooks',
                               'final-models',
                               'learningcurve-dnn-easy-final-features',
                              )

In [30]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.sigmoid
model_features.output_size = 1
model_features.metrics = ['mse']
model_features.learning_rate = model_features.learining_rate

## Train network

In [37]:
num_epochs = 200
patience = 10

model_id_save_as = 'dnn-easy'

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    patience=patience,
    mode='min',
    min_delta=0.01,
    restore_best_weights=True)

## Scale input data

In [38]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)

# Train

In [39]:
kf = KFold(n_splits=5)

k = 0
for train_index, validation_index in kf.split(training_spectra_scaled):
    model = compile_model(
        build_dnn_model,
        model_features)

    csv_logger = tf.keras.callbacks.CSVLogger(
        os.path.join('final-models-keras/',
        model_id_save_as+'_fold' + str(k) + '.log')

    output = model.fit(
        x=training_spectra_scaled[train_index],
        y=all_keys[train_index],
        epochs=num_epochs,
        verbose=1,
        validation_data=(training_spectra_scaled[validation_index],
                         all_keys[validation_index]),
        shuffle=True,
        callbacks=[earlystop_callback, csv_logger],
    )
    model.save(
        os.path.join('final-models-keras',
                     model_id_save_as + '_fold' + str(k) + '.hdf5')
    k += 1

Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 10s 127us/sample - loss: 0.3280 - mean_squared_error: 0.0886 - val_loss: 0.0922 - val_mean_squared_error: 0.0746
Epoch 2/200
80000/80000 [==============================] - 9s 115us/sample - loss: 0.0823 - mean_squared_error: 0.0720 - val_loss: 0.0741 - val_mean_squared_error: 0.0687
Epoch 3/200
80000/80000 [==============================] - 9s 118us/sample - loss: 0.0783 - mean_squared_error: 0.0717 - val_loss: 0.0696 - val_mean_squared_error: 0.0648
Epoch 4/200
80000/80000 [==============================] - 9s 118us/sample - loss: 0.0738 - mean_squared_error: 0.0690 - val_loss: 0.0677 - val_mean_squared_error: 0.0630
Epoch 5/200
80000/80000 [==============================] - 9s 118us/sample - loss: 0.0720 - mean_squared_error: 0.0676 - val_loss: 0.0634 - val_mean_squared_error: 0.0595
Epoch 6/200
80000/80000 [==============================] - 9s 119us/sample - loss: 0.0717 - me